In [4]:
import os
from typing import List, Dict, Optional

In [5]:
os.getcwd()

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin'

In [10]:
# test_dir = "../tests/img.test.dir/001-001/PAR REC"
test_dir = "../tests/img.test.dir"

In [11]:
os.listdir(test_dir)

['001-001',
 '002-001',
 '003-001',
 '900XXT5',
 '901XXP5',
 '902XXY8',
 'C01-001',
 'C02-001',
 'C03-001',
 'P01-001',
 'P02-001',
 'P03-001']

In [8]:
import utils.img_dir as id

In [16]:
dir_list,type_list = id.img_dir_list(test_dir)

In [17]:
dir_list

['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\DICOM\\20211701',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\NIFTI',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\001-001\\PAR REC',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\002-001\\DICOM\\20211701',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\002-001\\NIFTI',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\002-001\\PAR REC',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\003-001\\DICOM\\20211701',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\003-001\\NIFTI',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\tests\\img.test.dir\\003-001\\PAR REC',
 'c:\\User

In [1]:
# construct BIDS dictionary
# 
# BIDS dict
# 
# subject data dict
# metadata dict (should include parameter data)
# JSON file dict
# parameter dict: 
#   - dwi: acq, dir, run [, sbref]
#   - func: task, acq, dir, ce, echo, run, rec
#       - contrast_label: bold, cbv, phase [, sbref]
#   - anat: acq, dir, ce, rec, run
#       - modality: T1w, T2w, etc.
#   - fmap: 
# 
# Relevant links:
# https://bids-specification.readthedocs.io/en/stable/04-modality-specific-files/01-magnetic-resonance-imaging-data.html

In [3]:
class SubDataInfo():
    # place holder class
    # 
    # class implementation performed in another
    # jupyter notebook
    pass

In [8]:
def construct_bids_file(sub_data: SubDataInfo,
                        meta_dict: Optional[Dict] = None,
                        json_dict: Optional[Dict] = None
                        ):
    '''working doc-string'''

    bids_dict: Dict = {}

    if meta_dict:
        pass
    else:
        meta_dict: Dict = {}
    
    if json_dict:
        pass
    else:
        json_dict = {}
    
    tmp_dict: Dict = {"sub": sub_data.sub,
                      "ses": sub_data.ses }

    # bids_dict: Dict = multi_update(**meta_dict,
    #                                **json_dict,
    #                                **tmp_dict) # This function does not exist yet